Server

In [1]:
import socket
import random

# Global Constants
IP = "127.0.0.1"
PORT = 50003
FRAGMENT_SIZE = 500
WINDOW_SIZE = 10
RECEIVED_FILE_NAME = "received_sound_file.pdf"

# Error Simulation Switches
SIM_ACK_LOSS = True
SIM_PACKET_ORDER_MIX = True

# Global Variables
client_list = []
server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind((IP, PORT))

def establish_connection(client_addr):
    client_list.append(client_addr)
    server_socket.sendto(b"ACK", client_addr)
    print(f"Connection established with {client_addr[0]}:{client_addr[1]}")

def process_final_window_packet(client_addr, rec_data_buffer):
    # Simulate packet order randomization
    if SIM_PACKET_ORDER_MIX:
        random.shuffle(rec_data_buffer)

    # Save data to a file
    with open(RECEIVED_FILE_NAME, 'ab') as f:
        for data in rec_data_buffer:
            f.write(data)

    # Send ACK_ALL to the client
    if not SIM_ACK_LOSS:
        server_socket.sendto(b"ACK_ALL", client_addr)

def main():
    print("** UDP SERVER LOG **")
    print("Simulate ACK loss:", SIM_ACK_LOSS)
    print("Simulate packet order randomization:", SIM_PACKET_ORDER_MIX)
    print(f"Starting server on {IP}:{PORT}\n")

    while True:
        print("Waiting...")
        msg, client_addr = server_socket.recvfrom(FRAGMENT_SIZE)
        print(f"Message received from {client_addr[0]}:{client_addr[1]}: {msg.decode()}")

        if msg == b"CONN_REQ":
            establish_connection(client_addr)
        elif msg == b"REQ_TO_SEND" and client_addr in client_list:
            server_socket.sendto(b"ACK", client_addr)
            print("Authentication confirmed. Receiving file...")

            # Receive and process the file data here

            print("\nFile saved successfully!")

            # Close the server
            server_socket.close()
            print("Closing server...")
            break

if __name__ == "__main__":
    main()


** UDP SERVER LOG **
Simulate ACK loss: True
Simulate packet order randomization: True
Starting server on 127.0.0.1:50003

Waiting...
Message received from 127.0.0.1:57503: CONN_REQ
Connection established with 127.0.0.1:57503
Waiting...
Message received from 127.0.0.1:57503: DISCONNECT
Waiting...
Message received from 127.0.0.1:61976: CONN_REQ
Connection established with 127.0.0.1:61976
Waiting...
Message received from 127.0.0.1:61976: REQ_TO_SEND
Authentication confirmed. Receiving file...

File saved successfully!
Closing server...
